In [495]:
%load_ext autoreload
%autoreload 2
import bert
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import spike_queries
from termcolor import colored

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [455]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [493]:
def load_results(fname):
    
    with open(fname, "r", encoding = "utf-8") as f:
        sents = f.readlines()
    
    sents = [s.strip().split("\t")[-1] for s in sents]
    original, results = sents[0], sents[1:]
    return original, results


def get_spike_results_arguments_representations(model, spike_results, layers):
    
    sents = spike_results["sentence_text"].tolist()
    arg1_idx_start = spike_results["arg1_first_index"].to_numpy().astype(int)
    arg2_idx_start = spike_results["arg2_first_index"].to_numpy().astype(int)
    arg1_idx_end = spike_results["arg1_last_index"].to_numpy().astype(int)
    arg2_idx_end = spike_results["arg2_last_index"].to_numpy().astype(int)
    
    arg1_rep = []
    arg2_rep = []
    
    for s, arg1_start, arg2_start, arg1_end, arg2_end in zip(sents, arg1_idx_start, arg2_idx_start, arg1_idx_end, arg2_idx_end):
        #idx_to_mask = [arg1_start, arg2_start, arg1_end, arg2_end]
        H, _, _, orig2tok = model.encode(s, layers = layers)

        h1, h2 = H[orig2tok[arg1_start]:orig2tok[arg1_end] + 1], H[orig2tok[arg2_start]:orig2tok[arg2_end] + 1]
        
        h1 = np.mean(h1, axis = 0)
        h2 = np.mean(h2, axis = 0)
        
        arg1_rep.append(h1)
        arg2_rep.append(h2)
        
    arg1_mean = np.mean(arg1_rep, axis = 0)
    arg2_mean = np.mean(arg2_rep, axis = 0)
        
    return arg1_mean, arg2_mean
    

def print_sentence_nicely(sentence: str, ind1, ind2):
    
    arg1_sign = "**"
    arg2_sign = "++"
    arg1_color, arg2_color = "red", "blue"
    
    if not ind2 > ind1: 
        ind1, ind2 = ind2, ind1
        arg1_sign, arg2_sign = arg2_sign, arg1_sign
        arg1_color, arg2_color = arg2_color, arg1_color
        
    splitted = sentence.split(" ")
    before_arg1 = " ".join(splitted[:ind1])
    arg1 = splitted[ind1]
    arg2 = splitted[ind2]
    between = " ".join(splitted[ind1 + 1: ind2])
    suffix = " ".join(splitted[ind2+1:])
    
    return before_arg1 +  " " + arg1_sign + colored(arg1, arg1_color) + arg1_sign + " " + between + " " + arg2_sign + colored(arg2, arg2_color) + arg2_sign + " " + suffix



def main(filename, layers = [-1]):
    

    query, results1 = load_results(filename)
    spike_results = spike_queries.perform_query(query, dataset_name = "covid19", num_results = 50, query_type = "syntactic")
    spike_results = spike_results[spike_results['sentence_text'].notnull()]
    arg1_rep, arg2_rep = get_spike_results_arguments_representations(model, spike_results, layers)
    
    print(color.BOLD + "QUERY" + color.END + ":\n{}".format(query))
    
    first, first_ind1, first_ind2 = spike_results["sentence_text"].tolist()[-1], int(spike_results["arg1_first_index"].tolist()[-1]), int(spike_results["arg2_first_index"].tolist()[-1])
    print(color.BOLD + "\nFIRST SPIKE RESULT" + color.END + ":\n{}".format(print_sentence_nicely(first, first_ind1, first_ind2)))
    print(color.BOLD + "\nAUGMENTATION RESULTS:\n" + color.END)
    
    for s in results1:
        H, tokenized_text, tok_to_orig_map, orig2tok = model.encode(s, layers = layers)
        #print("H shape: {}".format(H.shape))
        sims_arg1 = cosine_similarity([arg1_rep], H[1:-2])[0]
        sims_arg2 = cosine_similarity([arg2_rep], H[1:-2])[0]
        arg1_ind = np.argmax(sims_arg1) + 1
        
        sims_arg2[np.argmax(sims_arg1)] = -1
        arg2_ind = np.argmax(sims_arg2) + 1
        #print(arg1_ind, arg2_ind)
        if arg1_ind not in tok_to_orig_map or arg2_ind not in tok_to_orig_map:
            print("ERROR")
            continue
        print(print_sentence_nicely(s, tok_to_orig_map[arg1_ind], tok_to_orig_map[arg2_ind]))
        print("-----------------------------------")
    
    print(color.BOLD + "=======================================================" + color.END)


In [457]:
model = bert.BertEncoder("cpu", "scibert")

loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/allenai/scibert_scivocab_uncased/config.json from cache at /home/shauli/.cache/torch/transformers/199e28e62d2210c23d63625bd9eecc20cf72a156b29e2a540d4933af4f50bda1.4b6b9f5d813f7395e7ea533039e02deb1723d8fd9d8ba655391a01a69ad6223d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 31090
}

loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/allenai/scibert_scivocab_uncased/config.json from cache at /home/shauli/.cache/torch/transformers/199e28e62d2210c23d63625bd9eecc20cf72a156b29e2a540d4933af4f50bda1.4b6b9f5d8

# Alignment Experiments

In [514]:
main("results1.txt", layers = [-1])

QUERY:
<>arg1:virus $infection $causes a <>arg2:condition .

FIRST SPIKE RESULT:
178 Thus , **leishmania** infection causes ++secondary++ infections , and activation/reactivation of dormant infections that can lead to severe or lethal outcomes through multiple mechanisms .

AUGMENTATION RESULTS:

In humans , **JEV** infection can cause Japanese encephalitis ( JE ) with ++severe++ central nervous system disorders .
-----------------------------------
Hepatotropic **virus** , like MHV-3 infection in mice , can induce exaggerated ++inflammation++ in the liver and cause life-threatening viral FH .
-----------------------------------
Human metapneumovirus ( hMPV ) **infection** causes respiratory tract ++disease++ similar to that observed during human respiratory syncytial virus infection ( hRSV ) .
-----------------------------------
In humans , CHIKV **infections** cause a debilitating ++disease++ with acute febrile illness and long-term polyarthralgia .
----------------------------------

Accuracy: 20/20

In [511]:
main("results2.txt", [-1])

QUERY:
infected arg1:patients $usually $develop <>arg2:[entity]respiratory illness.

FIRST SPIKE RESULT:
12 , 27 , [ 33 ] [ 34 ] [ 35 ] **Puppies** that are nursing from CHV-seronegative dams usually develop the fatal ++multisystemic++ disease , while puppies that suckle from CHV-seropositive dams remain asymptomatic but still become infected .

AUGMENTATION RESULTS:

After an incubation period of 5 to 14 days , SARS-CoV-2-infected **people** commonly manifest features of ++pneumonia++ , including fever , dry cough , dyspnoea , myalgia and fatigue .
-----------------------------------
After an incubation period of 2 to 3 days , **patients** who have pneumonic plague typically develop fulminant ++pneumonia++ , with malaise , high fever , cough , hemoptysis , and septicemia with ecchymoses and extremity necrosis .
-----------------------------------
 **Patients** suffering from severe DENV infection often exhibit encephalopathy and ++encephalitis++ .
-----------------------------------
F

Accuracy: 19/20

In [510]:
main("results3.txt", [-1])

QUERY:
a arg1:subset of patients  $progress to arg2:hemorrhagic fever

FIRST SPIKE RESULT:
The **disease** may progress across the bone of the skull base to the ++jugular++ foramen and hypoglossal canal , resulting in palsies of the ninth , tenth , eleventh and twelfth cranial nerves .

AUGMENTATION RESULTS:

 ++Severe++  **cases** may progress to a capillary leak syndrome with septic shock , rash , facial and neck swelling , and multi-organ system failure .
-----------------------------------
When symptoms occur , IPS **may** rapidly progress to ++pulmonary++ dysfunction requiring mechanical ventilation .
-----------------------------------
 ++Clinical++ signs are nonspecific in the early stages , but as the **disease** progresses to involve more of the liver and impair regeneration , icterus , ascites , and hepatic encephalopathy may develop as typical correlates with hepatic insufficiency .
-----------------------------------
The **patients** can develop radiographic findings of bil

Accuarcy: 13/20

In [509]:
main("results4.txt", [-1])

QUERY:
people with arg1:[e]asthma are $susceptible to arg2:[e]asthma

FIRST SPIKE RESULT:
However , children with inborn **errors** in TLR3 signaling ( 54 ) ( 55 ) ( 56 ) or defects resulting in abnormal signaling through several antiviral TLRs including 3 , 7 , 8 , 9 ( e.g. UNC-93B deficiency)(57 ) are susceptible to ++HSVE++ .

AUGMENTATION RESULTS:

Patients with lung **cancer** are highly susceptible to ++infection++ compared to healthy individuals because of systemic immunosuppression induced by malignancy and anticancer therapy .
-----------------------------------
Patients with lung **cancer** are highly susceptible to ++infection++ compared to healthy individuals because of systemic immunosuppression induced by malignancy and anticancer therapy .
-----------------------------------
 **Patients** with ARDS triggered by viral ++infection++ , in particular influenza , are prone to invasive aspergillosis even in absence of prior immunodeficiency .
----------------------------------

Accuracy: 16/20

In [508]:
main("results5.txt", [-1])

QUERY:
arg1:[e]paracetamol is $not useful for treating arg2:[e]asthma.

FIRST SPIKE RESULT:
There are cases of ADEM or even fulminant presentation such as AHL where **steroids** alone are not sufficient for suppressing ++inflammation++ and improving clinical findings .

AUGMENTATION RESULTS:

However , while tocilizumab is a promising agent against COVID-19 , **it** is not an appropriate agent in patients with active or latent ++tuberculosis++ , bacterial and fungal infections , multi-organ failure , and gastrointestinal perforation [ 7 ] .
-----------------------------------
There are cases of ADEM or even fulminant presentation such as AHL where **steroids** alone are not sufficient for suppressing ++inflammation++ and improving clinical findings .
-----------------------------------
Thus , **ribavirin** may not be useful for treating ++SARS++ infections because of its questionable efficacy and because of its known toxicity ( reviewed by van Vonderen et al. , 2003 ; Lai , 2005 ) .
--

Accuracy: 15/20

In [507]:
main("results6.txt", [-1])

QUERY:
arg1:[e]stroke is a $complication of :[e]COVID-19 arg2:infection

FIRST SPIKE RESULT:
In conclusion , hepatic **injury** is likely a complication of COVID-19 ++infection++ .

AUGMENTATION RESULTS:

Acute kidney **injury** is a possible manifestation in ++severe++ forms of COVID-19 , conferring a high mortality 7 .
-----------------------------------
Coagulation dysfunction is one of the major causes for **death** in ++patients++ with severe COVID-19 [ 4 ] .
-----------------------------------
ERROR
 **Hypoxia** is the most common manifestation of ++patients++ with COVID-19 , especially in severely or critically ill patients .
-----------------------------------
Early-onset **pneumonia** is a common and severe complication that is related to aspiration in ++patients++ with GBS .
-----------------------------------
 **Barotrauma** is a common complication in ++patients++ with SARS .
-----------------------------------
COVID-19 **infection**  ++is++ associated with several cardiac 

Accuracy: 16/20

In [505]:
main("results7.txt", [-1])

QUERY:
the recommended arg1:[w]quarantine period is arg2:14 :[w]days

FIRST SPIKE RESULT:
The CERs of interventions was unstable when the **quarantine** delay-time was no less than ++five++ days ( figure 2d ) .

AUGMENTATION RESULTS:

If we aim to control the failure rate of quarantine to be below 1 % with 95 % confidence , then the **quarantine** period must be at least ++22++ days .
-----------------------------------
In general , the duration of the **quarantine** period should be 21 to ++30++ days .
-----------------------------------
Although for extreme cases , the **quarantine** period should be extended up to ++three++ weeks .
-----------------------------------
At the beginning and the end of a SARS epidemic , to control the epidemic completely , we recommend a period of ++22++ days for **quarantine** , which would capture 99 percent of all probable SARS cases with a probability of 0.9 .
-----------------------------------
Based on the 95th percentile estimate of the incubatio

Accuracy: 19/20

In [504]:
main("results8.txt", [-1])

QUERY:
arg1:[e]COVID-19 $infects arg2:cells

FIRST SPIKE RESULT:
Besides the lung , the **SARS-CoV-2** virus also infects the ++GI++ tract , causing the patients also to experience diarrhea .

AUGMENTATION RESULTS:

 **SARS-CoV-2** infects ACE2positive ++cells++ in the oral mucosa and lungs , including ACE2 + AT2 cells in the alveoli .
-----------------------------------
SARS-CoV targets human angiotensin-converting enzyme **2** ( hACE2 ) and infects intrapulmonary epithelial ++cells++ more than cells of the upper airways [ 13 , 14 ] .
-----------------------------------
Initially , **SARS-CoV-2** infects ++cells++ in the respiratory system and causes inflammation and cell death .
-----------------------------------
 **SARS-CoV-2** infects ACE2 + ++cells++ in the oral mucosa and lungs , including ACE-2 cells in the alveoli [ 97 ] .
-----------------------------------
In the lungs , **SARS-CoV-2** infects type I and type II alveolar epithelial ++cells++ , as well as alveolar macrophages

Accuracy: 17/20

In [503]:
main("results9.txt", [-1])

QUERY:
arg1:[e]COVID-19 activates the arg2:ATP $receptor

FIRST SPIKE RESULT:
 **SARS-CoV-2** uses the angiotensin converting ++enzyme++ 2 ( ACE2 ) receptor to enter the host and this receptor is highly expressed in both the respiratory and gastrointestinal tract 9 , 10 , 11 .

AUGMENTATION RESULTS:

 **SARS-CoV-2** enters respiratory epithelial cells by attaching to angiotensin converting enzyme-2 ( ACE-2 ) via S-protein ; ++ACE-2++ is also a receptor for SARS-CoV-1 [ 27 ] .
-----------------------------------
Presumably , SARS-CoV infects cells through the S protein , which binds to cell surface receptor-angiotensin-converting ++enzyme++  **2** [ 4 ] .
-----------------------------------
Human angiotensin-converting enzyme 2 ( ++ACE2++ ) is a functional receptor hijacked by **SARS-CoV-2** for cell entry , similar to SARS-CoV [ 8 , 16 ] .
-----------------------------------
It is now known that **SARS-CoV-2** utilizes the same receptor of SARS-CoV , which is angiotensin converting enz

Accuracy: 19/20

In [502]:
main("results10.txt", [-1])

QUERY:
low blood oxygen arg1:levels are a $risk factor in arg2:[e]COVID-19 infection

FIRST SPIKE RESULT:
Among all univariable parameters , **APACHE** II , SOFA , lymphocytes , CRP , LDH , AST , cTnI , BNP , et al were significantly independent risk factors of ++COVID-19++ severity .

AUGMENTATION RESULTS:

Similar to other studies ， our data indicated senior and hypertension are the high risk factors for severe ++COVID-19++  **infection** .
-----------------------------------
These findings suggested that **men** with severe ++COVID-19++ is susceptible to secondary infections with virus or bacteria , resulting in higher utilization rate of advanced antiviral therapy and antibiotics .
-----------------------------------
In this study , preceding RVIs was one of the independent risk factors associated with severe ++pneumococcal++  **pneumonia** ( a higher PSI score ≥91 ) , suggesting a potential role of preceding RVIs on clinical severity .
-----------------------------------
Our study

Accuarrcy: 12/20